# Zeitliche Animation und Heatmaps von Flüchtlingsbewegungen

Dieses Notebook erweitert die statische Analyse um zeitliche Visualisierungen.
Es erstellt animierte Flowmaps, bei denen Flüchtlingsbewegungen als bewegte Partikel dargestellt werden, sowie animierte Heatmaps, die für jedes Jahr die Zu- und Abwanderung pro Land zeigen.
Dadurch werden langfristige Entwicklungen und globale Muster der Flüchtlingsbewegungen anschaulich über die Zeit hinweg visualisiert.


Dieser Abschnitt lädt die Welt-Geodaten sowie die Flussdaten für alle Jahre.
Zusätzlich werden sehr kleine geografische Objekte entfernt und die verfügbaren Jahre bestimmt, um spätere Animationen korrekt über die Zeit steuern zu können.


In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib as mpl
from IPython.display import HTML

world_path = r"additional_data/custom.geo.json"
world = gpd.read_file(world_path)

if "name" in world.columns:
    world = world.rename(columns={"name": "country"})
elif "admin" in world.columns:
    world = world.rename(columns={"admin": "country"})
elif "ADMIN" in world.columns:
    world = world.rename(columns={"ADMIN": "country"})

world = world[
    [len(geom.exterior.coords) > 10 if geom.geom_type == "Polygon" else True
     for geom in world.geometry]
]

flows_path = r"output_csv_files/flows_with_coords_all_years.csv"
flows_all = pd.read_csv(flows_path)

flows_all["Year"] = flows_all["Year"].astype(int)
flows_all["Value"] = pd.to_numeric(flows_all["Value"], errors="coerce")

flows_all = flows_all.dropna(subset=[
    "Value", "origin_lon", "origin_lat", "asylum_lon", "asylum_lat"
])

years = sorted(flows_all["Year"].unique())


Diese Funktion wandelt aggregierte Fluchtbewegungen in einzelne Partikel um, wobei eine bestimmte Anzahl von Personen durch einen Punkt repräsentiert wird.  
Zur besseren visuellen Trennung werden Start- und Zielpunkte leicht zufällig verschoben, sodass die spätere Animation flüssig und übersichtlich dargestellt werden kann.








In [ ]:
def build_particles(flows,
                    person_unit=1000,
                    max_flows=200,
                    max_people_per_flow=15,
                    random_state=42):

    rng = np.random.default_rng(random_state)

    flows = flows.sort_values("Value", ascending=False).head(max_flows)

    sx, sy, ex, ey = [], [], [], []

    for _, row in flows.iterrows():
        value = row["Value"]
        n_people = int(value // person_unit)

        if n_people <= 0:
            continue

        n_people = min(n_people, max_people_per_flow)

        ox, oy = row["origin_lon"], row["origin_lat"]
        ax_, ay_ = row["asylum_lon"], row["asylum_lat"]

        jitter = 0.5
        sx.extend(ox + rng.normal(0, jitter, n_people))
        sy.extend(oy + rng.normal(0, jitter, n_people))
        ex.extend(ax_ + rng.normal(0, jitter, n_people))
        ey.extend(ay_ + rng.normal(0, jitter, n_people))

    return np.array(sx), np.array(sy), np.array(ex), np.array(ey)


Diese Funktion erzeugt eine zeitliche Animation, in der Flüchtlingsbewegungen als bewegte Partikel zwischen Herkunfts- und Aufnahmeländern dargestellt werden, wodurch kontinuierliche und gut nachvollziehbare Migrationsbewegungen visualisiert werden.

In [ ]:
def animate_flows(world,
                  flows,
                  year,
                  title,
                  person_unit=1000,
                  max_flows=200,
                  max_people_per_flow=15,
                  frames_per_cycle=80,
                  cycles=3,
                  marker_size=10):

    sx, sy, ex, ey = build_particles(
        flows,
        person_unit=person_unit,
        max_flows=max_flows,
        max_people_per_flow=max_people_per_flow
    )

    if len(sx) == 0:
        print("Keine Partikel erzeugt.")
        return None

    total_frames = frames_per_cycle * cycles

    fig, ax = plt.subplots(figsize=(22, 12))
    world.plot(ax=ax, color="#E6E6E6", edgecolor="#BBBBBB", linewidth=0.5)

    scat = ax.scatter(sx, sy, s=marker_size, color="#cc0000", alpha=0.8)

    ax.set_title(title, fontsize=26)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.grid(color="#DDDDDD", linestyle="--", linewidth=0.3)

    def update(frame):
        t = (frame % frames_per_cycle) / frames_per_cycle
        x = sx + t * (ex - sx)
        y = sy + t * (ey - sy)
        scat.set_offsets(np.column_stack([x, y]))
        return scat,

    anim = FuncAnimation(
        fig,
        update,
        frames=total_frames,
        interval=50,
        blit=True
    )

    plt.close(fig)
    return anim


In [ ]:
mpl.rcParams["animation.embed_limit"] = 300

YEAR = 2016
flows_year = flows_all[flows_all["Year"] == YEAR]

anim = animate_flows(
    world,
    flows_year,
    year=YEAR,
    title="Globale Flüchtlingsbewegungen (2016)",
    person_unit=5000,
    max_flows=1000,
    max_people_per_flow=15,
    frames_per_cycle=80,
    cycles=2,
    marker_size=12
)

if anim is not None:
    anim.save(
        "output_gifs/global_flowmap_2016.gif",
        writer="pillow",
        fps=10
    )


Dieser Abschnitt aggregiert die Flussdaten nach Jahr und Aufnahmeland und berechnet die gesamte Anzahl aufgenommener Flüchtlinge pro Land, um die Grundlage für zeitliche Heatmap-Visualisierungen zu schaffen.

In [ ]:
# %%
inflow_yearly = (
    flows_all
    .groupby(["Year", "Asylum_clean"])["Value"]
    .sum()
    .reset_index()
    .rename(columns={
        "Asylum_clean": "country",
        "Value": "refugee_share"
    })
)


Dieser Abschnitt erzeugt eine animierte Weltkarte, die für jedes Jahr die Gesamtzahl der aufgenommenen Flüchtlinge pro Land farblich darstellt.  
Die Farbskala bleibt über alle Jahre konstant, sodass zeitliche Veränderungen der Aufnahmestärke vergleichbar und visuell nachvollziehbar sind.

In [ ]:
vmin = inflow_yearly["refugee_share"].min()
vmax = inflow_yearly["refugee_share"].max()

norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
cmap = plt.cm.Reds

fig, ax = plt.subplots(figsize=(24, 12))

sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])

cbar = fig.colorbar(sm, ax=ax, fraction=0.03, pad=0.02)
cbar.set_label("Number of Refugees", fontsize=16)

def update(year):
    ax.clear()

    data_year = inflow_yearly[inflow_yearly["Year"] == year]
    merged = world.merge(data_year, on="country", how="left")

    merged.plot(
        column="refugee_share",
        cmap="Reds",
        linewidth=0.5,
        edgecolor="black",
        ax=ax,
        legend=False,
        missing_kwds={"color": "white", "edgecolor": "black"}
    )

    ax.set_title(
        f"Total Refugee Inflows per Country ({year})",
        fontsize=32
    )
    ax.axis("off")

anim = FuncAnimation(fig, update, frames=years, interval=500)
anim.save("output_gifs/inflow_heatmap.gif", writer="pillow", fps=4)
plt.close(fig)


Dieser Abschnitt aggregiert die Flussdaten nach Jahr und ausgebendem Lanf und berechnet die gesamte Anzahl aufgenommener Flüchtlinge pro Land, um die Grundlage für zeitliche Heatmap-Visualisierungen zu schaffen.

In [ ]:
outflow_yearly = (
    flows_all
    .groupby(["Year", "Origin_clean"])["Value"]
    .sum()
    .reset_index()
    .rename(columns={
        "Origin_clean": "country",
        "Value": "refugee_share"
    })
)


Dieser Abschnitt erstellt eine animierte Weltkarte, die für jedes Jahr die Gesamtzahl der ausgehenden Flüchtlinge pro Land visualisiert.  
Analog zur Inflow-Heatmap wird eine konstante Farbskala verwendet, um zeitliche Veränderungen der Abwanderungsintensität klar vergleichen zu können.

In [ ]:
vmin = outflow_yearly["refugee_share"].min()
vmax = outflow_yearly["refugee_share"].max()

norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

fig, ax = plt.subplots(figsize=(24, 12))

sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])

cbar = fig.colorbar(sm, ax=ax, fraction=0.03, pad=0.02)
cbar.set_label("Number of Refugees", fontsize=16)

def update(year):
    ax.clear()

    data_year = outflow_yearly[outflow_yearly["Year"] == year]
    merged = world.merge(data_year, on="country", how="left")

    merged.plot(
        column="refugee_share",
        cmap="Reds",
        linewidth=0.5,
        edgecolor="black",
        ax=ax,
        legend=False,
        missing_kwds={"color": "white", "edgecolor": "black"}
    )

    ax.set_title(
        f"Total Refugee Outflows per Country ({year})",
        fontsize=32
    )
    ax.axis("off")

anim = FuncAnimation(fig, update, frames=years, interval=500)
anim.save("output_gifs/outflow_heatmap.gif", writer="pillow", fps=4)
plt.close(fig)
